# От множества к пространству

Чем «пространство» отличается от «множества»? Если совсем на пальцах. Тем, что над его элементами определены операции. Соответственно, если мы просто «напишем» класс, то определим тип данных, элемент множества. А если реализуем для него операции — определим пространство.

Тяжело найти что-то, чего нет в стандартной библиотеке. А тем более во внешних...

![](https://imgs.xkcd.com/comics/python.png)

Давайте считать, что кольцо многочленов не реализовано. А ведь полезная вещь! А как его реализовать? А очень просто: [вот тут всё написано](https://docs.python.org/3.7/reference/datamodel.html#emulating-numeric-types)!

In [2]:
from numbers import Number
import itertools
import copy


class MatrixError(ValueError):
    pass


class Matrix:
    """ """
    
    def __init__(self, arg: 'list[list[Number]]|Matrix'= 0):
        """
        """
        if isinstance(arg, Matrix):
            self.mat = copy.deepcopy(arg.mat)
        elif isinstance(arg, list):
            for i in range(1, len(arg)):
                if not isinstance(arg[i-1], list) and not isinstance(arg[i], list) and len(arg[i-1]) != len(arg[i]):
                    raise MatrixError("You are trying to create matrix from " + repr(arg))
            self.mat = copy.deepcopy(arg)
        else:
            raise MatrixError("You are trying to create Matrix from " + repr(arg))

        self.cleanup(self.mat)

        
    @staticmethod
    def cleanup(mat: 'list[list[Number]]') -> 'None':
        epsilon = 1e-25
        for i in range(len(mat)):
            for j in range(len(mat[i])):
                if abs(mat[i][j]) < epsilon:
                    mat[i][j] = 0
        
    def __str__(self) -> 'str':
        return "[" + "\n ".join([str(row) for row in self.mat]) + "]"
    
    def __eq__(self, other: 'Matrix') -> 'bool':
        if isinstance(other, Matrix):
            return self.mat == other.mat
        else:
            raise MatrixError("Can't say if Matrix is equal to " + str(type(other)))
            
    
    def __add__(self, other: 'Matrix') -> 'Matrix':
        if not isinstance(other, Matrix):
            raise MatrixError("Can't add not Matrix " + str(type(other)))
            
        if (len(self.mat) != len(other.mat) or len(self.mat[0]) != len(other.mat[0])):
            raise MatrixError("Can't add different size Matrix " + str(type(other)))
        
        res = copy.deepcopy(self.mat)
        for i in range(len(res)):
            for j in range (len(res[i])):
                res[i][j] += other.mat[i][j]
        
        return Matrix(res)
        
        
    def __radd__(self, other: 'Matrix') -> 'Matrix':
        return self.__add__(other)  # Коммутативность

    
    def __neg__(self) -> 'Matrix':
        return Matrix([[-c for c in row] for row in self.mat])

    
    def __sub__(self, other: 'Matrix') -> 'Matrix':
        if not isinstance(other, Matrix):
            raise MatrixError("You are trying to subtract not Matrix " + str(type(other)))

        return self.__add__(other.__neg__())

    
    def __rsub__(self, other: 'Matrix') -> 'Matrix':
        return self.__neg__().__add__(other)

    
    def __mul__(self, a: 'Number') -> 'Matrix':
        if not isinstance(a, Number):
            raise MatrixError("You are trying to multiply not on number " + str(type(a)))
            
        return Matrix([[c*a for c in row] for row in self.mat])
        
        
    def __rmul__(self, other: 'Number') -> 'Matrix':
        return self.__mul__(other)

Ну а теперь давайте посмотрим его в деле!

In [3]:
m1 = Matrix([[1,3],[2,4]])
m2 = Matrix([[3,3],[4,5]])
m3 = Matrix([[4,5,6],[2,3,4]])

print(m1)
print(m2)
print(m3)
print(m1 == m2)

print(m1 + m2)
print(m1*5)
print(m2 - m1)
print(-m1)

[[1, 3]
 [2, 4]]
[[3, 3]
 [4, 5]]
[[4, 5, 6]
 [2, 3, 4]]
False
[[4, 6]
 [6, 9]]
[[5, 15]
 [10, 20]]
[[2, 0]
 [2, 1]]
[[-1, -3]
 [-2, -4]]


## Задание

Придумать и реализовать какой-нибудь свой тип данных с операциями. Вот идеи:

* числа по модулю;
* обычные числа, но с ошибками при вычислениях;
* [кватеринионы](https://en.wikipedia.org/wiki/Quaternion);
* можно матрицы (в т.ч. вектора) — они и так в NumPy есть, но всё равно же интересно;
* да тот же многочлен, но при помощи [`defaultdict`](https://docs.python.org/3.7/library/collections.html#collections.defaultdict) — ему не повредит;
* что угодно на свой вкус, над чем можно определить арифметику или её подобие.